# RNN from scratch to generate new text given a starting character


In [1]:
data = open('/home/payas/dl_box/data/GOT_books/001ssb.txt','r').read()

chars = list(set(data))
data_size, vocab_size =  len(data), len(chars)

print(data_size,vocab_size)


1607894 78


In [2]:
char_to_ix = {ch:i for i,ch in enumerate(chars)}
ix_to_char = {i:ch for i,ch in enumerate(chars)}
print(char_to_ix,ix_to_char)

{'Y': 0, '\n': 1, 'g': 2, 'c': 3, '"': 4, '-': 5, 'G': 6, ',': 7, '2': 8, 'z': 9, 'C': 10, '4': 11, 'K': 12, '6': 13, '`': 14, 'l': 15, '3': 16, '.': 17, 'I': 18, 'D': 19, 'i': 20, 's': 21, 'H': 22, '!': 23, 'm': 24, 'B': 25, ']': 26, 'b': 27, 'v': 28, 'q': 29, '1': 30, '8': 31, 'u': 32, '*': 33, 'f': 34, 't': 35, ')': 36, 'p': 37, 'x': 38, 'V': 39, 'e': 40, 'k': 41, 'P': 42, 'L': 43, 'w': 44, '?': 45, 'Q': 46, '(': 47, 'y': 48, '~': 49, 'h': 50, 'N': 51, ':': 52, '9': 53, 'S': 54, 'U': 55, 'A': 56, 'T': 57, 'M': 58, 'E': 59, 'F': 60, ' ': 61, "'": 62, ';': 63, 'J': 64, '0': 65, '5': 66, 'o': 67, 'a': 68, 'd': 69, 'R': 70, 'n': 71, 'O': 72, '7': 73, 'W': 74, 'X': 75, 'r': 76, 'j': 77} {0: 'Y', 1: '\n', 2: 'g', 3: 'c', 4: '"', 5: '-', 6: 'G', 7: ',', 8: '2', 9: 'z', 10: 'C', 11: '4', 12: 'K', 13: '6', 14: '`', 15: 'l', 16: '3', 17: '.', 18: 'I', 19: 'D', 20: 'i', 21: 's', 22: 'H', 23: '!', 24: 'm', 25: 'B', 26: ']', 27: 'b', 28: 'v', 29: 'q', 30: '1', 31: '8', 32: 'u', 33: '*', 34: 'f',

In [3]:
import numpy as np

vector_for_char_a = np.zeros((vocab_size,1))
vector_for_char_a[char_to_ix['s']] = 1
print(vector_for_char_a.ravel())

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.]


In [4]:
hidden_size = 500
seq_length = 25
learning_rate = 0.03


In [5]:
Wxh = np.random.randn(hidden_size, vocab_size) * 0.01
Whh = np.random.randn(hidden_size, hidden_size) * 0.01
Why = np.random.randn(vocab_size, hidden_size) * 0.01 

bh = np.zeros ((hidden_size,1))
by = np.zeros((vocab_size,1))



In [6]:
def lossFun(inputs, targets, hprev):
    
    xs, hs, ys, ps = {}, {}, {}, {}
    
    hs[-1] = np.copy(hprev)
    loss = 0 
    
    #Forward
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size,1))
        xs[t][inputs[t]] = 1 
        
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]), + bh)
        ys[t] = np.dot(Why, hs[t]) + by
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t]))
        
        loss += -np.log(ps[t][targets[t],0])
        
    #Backward
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    for t in reversed(range(len(inputs))):
        #output probabilities
        dy = np.copy(ps[t])
        #derive our first gradient
        dy[targets[t]] -= 1 # backprop into y  
        #compute output gradient -  output times hidden states transpose
        #When we apply the transpose weight matrix,  
        #we can think intuitively of this as moving the error backward
        #through the network, giving us some sort of measure of the error 
        #at the output of the lth layer. 
        #output gradient
        dWhy += np.dot(dy, hs[t].T)
        #derivative of output bias
        dby += dy
        #backpropagate!
        dh = np.dot(Why.T, dy) + dhnext # backprop into h                                                                                                                                         
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity                                                                                                                     
        dbh += dhraw #derivative of hidden bias
        dWxh += np.dot(dhraw, xs[t].T) #derivative of input to hidden layer weight
        dWhh += np.dot(dhraw, hs[t-1].T) #derivative of hidden layer to hidden layer weight
        dhnext = np.dot(Whh.T, dhraw) 
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients                                                                                                                 
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [7]:
#prediction, one full forward pass
def sample(h, seed_ix, n):
    """                                                                                                                                                                                         
    sample a sequence of integers from the model                                                                                                                                                
    h is memory state, seed_ix is seed letter for first time step   
    n is how many characters to predict
    """
    #create vector
    x = np.zeros((vocab_size, 1))
    #customize it for our seed char
    x[seed_ix] = 1
    #list to store generated chars
    ixes = []
    #for as many characters as we want to generate
    for t in range(n):
        #a hidden state at a given time step is a function 
        #of the input at the same time step modified by a weight matrix 
        #added to the hidden state of the previous time step 
        #multiplied by its own hidden state to hidden state matrix.
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        #compute output (unnormalised)
        y = np.dot(Why, h) + by
        ## probabilities for next chars
        p = np.exp(y) / np.sum(np.exp(y))
        #pick one with the highest probability 
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        #create a vector
        x = np.zeros((vocab_size, 1))
        #customize it for the predicted char
        x[ix] = 1
        #add it to the list
        ixes.append(ix)

    txt = ''.join(ix_to_char[ix] for ix in ixes)
    print (txt)
hprev = np.zeros((hidden_size,1)) # reset RNN memory  
#predict the 200 next characters given 'a'
sample(hprev,char_to_ix['a'],200)

`PG5k`by23I7(,kGg5.S0Yy5snfead
u)CSl!vIzyK(;oX8
Hnv28GcE]6DtzSld!uqu
Ig4';8O,W:
d
w..'T8C)JxypYqqlmtK0oifc'yy(,M~T;L~1o6'cXQ4HC8q;(6tXcgp92!5)up0*,SVv4IyQ6pkV',cCSF44qo-BEqako"(qMA5wS5ruCbSmLx:e-IoNt!


In [8]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad                                                                                                                
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0                                                                                                                        
while n<=1000*100:
    # prepare inputs (we're sweeping from left to right in steps seq_length long)
    # check "How to feed the loss function to see how this part works
    if p+seq_length+1 >= len(data) or n == 0:
        hprev = np.zeros((hidden_size,1)) # reset RNN memory                                                                                                                                      
        p = 0 # go from start of data                                                                                                                                                             
    
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

    # forward seq_length characters through the net and fetch gradient                                                                                                                          
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001

    # sample from the model now and then                                                                                                                                                        
    if n % 1000 == 0:
        print ('iter:',n) 
        print("loss:",smooth_loss) # print progress
        

    # perform parameter update with Adagrad                                                                                                                                                     
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by],
                                [dWxh, dWhh, dWhy, dbh, dby],
                                [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update                                                                                                                   

    p += seq_length # move data pointer                                                                                                                                                         
    n += 1 # iteration counter


iter: 0
loss: 108.917709635
iter: 1000
loss: 99.5320405749
iter: 2000
loss: 86.0123841408
iter: 3000
loss: 77.2816506481
iter: 4000
loss: 71.2628048417
iter: 5000
loss: 67.5329779409
iter: 6000
loss: 64.803788743
iter: 7000
loss: 62.6977612804
iter: 8000
loss: 61.6331194367
iter: 9000
loss: 60.7494459572
iter: 10000
loss: 60.3593812701
iter: 11000
loss: 60.0039702702
iter: 12000
loss: 59.2592127204
iter: 13000
loss: 58.6127187728
iter: 14000
loss: 58.3662676174
iter: 15000
loss: 57.5850812703
iter: 16000
loss: 57.6029838064
iter: 17000
loss: 56.7388483592
iter: 18000
loss: 56.381836564
iter: 19000
loss: 56.2219393161
iter: 20000
loss: 56.3734279244
iter: 21000
loss: 56.3098198681
iter: 22000
loss: 55.9952494477
iter: 23000
loss: 56.0656037934
iter: 24000
loss: 55.7120305475
iter: 25000
loss: 55.5598935273
iter: 26000
loss: 55.5984610432
iter: 27000
loss: 55.0856424727
iter: 28000
loss: 54.7568532381
iter: 29000
loss: 54.7311960699
iter: 30000
loss: 54.5738694943
iter: 31000
loss: 54.82

In [18]:
sample(hprev, char_to_ix['A'], 400)

 lorh thh  malt. 
ss when wes was to rept of to s ar mwagl. Lan Misan wowning urour to com 
tee of to the ondse the s aded abothed dine of thes be forne the pe the alpy?" him din jo annisan hos the s cepp t heol title Gre dee thole the lees, na the yous acl-cearl sin as st wre led. Drior oll diens and tore he set cal. the weees, a ke st jee, ly rom saire. He kine sa blete not Setey as if hivs na t



# code inspired by siraj raval's video.
